# Real Data Figures for Harmonic Field Model Paper

This notebook generates figures using real EEG/MEG data from MNE-Python's built-in datasets.

**Requirements:**
- `mne` (with datasets)
- `numpy`
- `scipy`
- `matplotlib`

Install with: `pip install mne numpy scipy matplotlib`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg, signal
from pathlib import Path

import mne
from mne.datasets import sample, sleep_physionet

# Set style for publication-quality figures
plt.rcParams.update({
    'font.size': 10,
    'font.family': 'serif',
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'axes.spines.top': False,
    'axes.spines.right': False,
})

# Output directory
OUTPUT_DIR = Path('.')

print(f"MNE version: {mne.__version__}")

## 1. Helper Functions for Harmonic Analysis

In [ ]:
def compute_sensor_laplacian(info, method='distance'):
    """
    Compute a graph Laplacian from sensor positions.
    
    Parameters
    ----------
    info : mne.Info
        MNE info object containing sensor positions
    method : str
        'distance' for distance-based connectivity
        
    Returns
    -------
    L : ndarray
        Graph Laplacian matrix
    pos_3d : ndarray
        3D sensor positions
    """
    # Get sensor positions
    pos_3d = np.array([ch['loc'][:3] for ch in info['chs'] 
                       if ch['kind'] == mne.io.constants.FIFF.FIFFV_EEG_CH 
                       or ch['kind'] == mne.io.constants.FIFF.FIFFV_MEG_CH])
    
    n_sensors = len(pos_3d)
    
    # Compute distance matrix
    dist = np.zeros((n_sensors, n_sensors))
    for i in range(n_sensors):
        for j in range(n_sensors):
            dist[i, j] = np.linalg.norm(pos_3d[i] - pos_3d[j])
    
    # Convert to adjacency (Gaussian kernel)
    sigma = np.median(dist[dist > 0])
    A = np.exp(-dist**2 / (2 * sigma**2))
    np.fill_diagonal(A, 0)
    
    # Compute Laplacian
    D = np.diag(A.sum(axis=1))
    L = D - A
    
    return L, pos_3d


def compute_eigenmodes(L):
    """
    Compute eigenmodes of the Laplacian.
    
    Returns
    -------
    eigenvalues : ndarray
        Sorted eigenvalues
    eigenvectors : ndarray
        Corresponding eigenvectors (columns)
    """
    eigenvalues, eigenvectors = linalg.eigh(L)
    idx = np.argsort(eigenvalues)
    return eigenvalues[idx], eigenvectors[:, idx]


def project_to_modes(data, eigenvectors):
    """
    Project data onto harmonic eigenmodes.
    
    Parameters
    ----------
    data : ndarray, shape (n_channels, n_times)
        Sensor data
    eigenvectors : ndarray, shape (n_channels, n_modes)
        Eigenmode basis
        
    Returns
    -------
    mode_amplitudes : ndarray, shape (n_modes, n_times)
        Mode amplitude time series
    """
    return eigenvectors.T @ data


def compute_mode_power(mode_amplitudes, normalize=True):
    """
    Compute power in each mode.
    
    Parameters
    ----------
    mode_amplitudes : ndarray, shape (n_modes, n_times)
        Mode amplitude time series
    normalize : bool
        If True, normalize to sum to 1
        
    Returns
    -------
    power : ndarray, shape (n_modes,)
        Power in each mode
    """
    power = np.mean(mode_amplitudes**2, axis=1)
    if normalize:
        power = power / power.sum()
    return power


def compute_mode_entropy(power):
    """Compute Shannon entropy of mode power distribution."""
    power = power[power > 0]  # Avoid log(0)
    return -np.sum(power * np.log(power))


def compute_participation_ratio(power):
    """Compute participation ratio (effective number of modes)."""
    return 1.0 / np.sum(power**2)

## 2. Load MNE Sample Dataset (MEG/EEG)

In [ ]:
# Download sample data (if not already downloaded)
print("Loading MNE sample dataset...")
data_path = sample.data_path()
raw_fname = data_path / 'MEG' / 'sample' / 'sample_audvis_raw.fif'

# Load raw data
raw = mne.io.read_raw_fif(raw_fname, preload=True, verbose=False)

# Pick EEG channels for simplicity
raw.pick_types(meg=False, eeg=True, stim=False, eog=False, exclude='bads')

# Filter
raw.filter(l_freq=0.5, h_freq=45, verbose=False)

print(f"Loaded {len(raw.ch_names)} EEG channels")
print(f"Sampling rate: {raw.info['sfreq']} Hz")
print(f"Duration: {raw.times[-1]:.1f} seconds")

## 3. Figure 5: Real EEG Harmonic Decomposition

In [ ]:
print("Generating Figure 5: Real EEG harmonic decomposition...")

# Compute sensor Laplacian
L, pos_3d = compute_sensor_laplacian(raw.info)
eigenvalues, eigenvectors = compute_eigenmodes(L)

# Get data segment
data, times = raw[:, :int(10 * raw.info['sfreq'])]  # 10 seconds

# Project to modes
mode_amplitudes = project_to_modes(data, eigenvectors)
power = compute_mode_power(mode_amplitudes)

# Create figure
fig, axes = plt.subplots(2, 3, figsize=(12, 7))

# Panel A: Sensor layout (2D projection)
ax = axes[0, 0]
pos_2d = pos_3d[:, :2]  # Project to 2D
ax.scatter(pos_2d[:, 0], pos_2d[:, 1], c='steelblue', s=50, alpha=0.7)
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title('(A) EEG Sensor Layout')
ax.set_aspect('equal')

# Panel B: Eigenvalue spectrum
ax = axes[0, 1]
ax.bar(range(len(eigenvalues)), eigenvalues, color='steelblue', alpha=0.7)
ax.set_xlabel('Mode index $k$')
ax.set_ylabel('Eigenvalue $\\lambda_k$')
ax.set_title('(B) Laplacian Eigenspectrum')

# Panel C: First few eigenmodes on sensor layout
ax = axes[0, 2]
mode_idx = 1  # First non-trivial mode
mode = eigenvectors[:, mode_idx]
scatter = ax.scatter(pos_2d[:, 0], pos_2d[:, 1], c=mode, cmap='RdBu_r', s=80,
                     vmin=-np.max(np.abs(mode)), vmax=np.max(np.abs(mode)))
plt.colorbar(scatter, ax=ax, label='Mode amplitude')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_title(f'(C) Mode $\\psi_1$ (eigenvalue={eigenvalues[mode_idx]:.2f})')
ax.set_aspect('equal')

# Panel D: Mode power distribution
ax = axes[1, 0]
ax.bar(range(len(power)), power, color='coral', alpha=0.7)
ax.set_xlabel('Mode index $k$')
ax.set_ylabel('Normalized power $p_k$')
ax.set_title('(D) Mode Power Distribution')

# Compute metrics
H = compute_mode_entropy(power)
PR = compute_participation_ratio(power)
H_max = np.log(len(power))
ax.text(0.95, 0.95, f'$H_{{mode}} = {H:.2f}$\n$H_{{max}} = {H_max:.2f}$\n$PR = {PR:.1f}$',
        transform=ax.transAxes, ha='right', va='top',
        fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

# Panel E: Mode amplitude time series
ax = axes[1, 1]
for k in range(min(5, len(mode_amplitudes))):
    ax.plot(times[:500], mode_amplitudes[k, :500] + k*5e-5, 
            label=f'Mode {k}', alpha=0.8)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Mode amplitude (offset)')
ax.set_title('(E) Mode Amplitude Time Series')
ax.legend(loc='upper right', fontsize=8)

# Panel F: Power spectral density of mode amplitudes
ax = axes[1, 2]
for k in [0, 1, 5, 10]:
    if k < len(mode_amplitudes):
        f, psd = signal.welch(mode_amplitudes[k], fs=raw.info['sfreq'], nperseg=512)
        ax.semilogy(f, psd, label=f'Mode {k}', alpha=0.8)
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title('(F) Mode Power Spectra')
ax.set_xlim(0, 45)
ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'fig5_real_eeg_harmonics.png')
plt.show()
print("Saved: fig5_real_eeg_harmonics.png")

## 4. Load Sleep Physionet Dataset

In [ ]:
print("Loading Sleep Physionet dataset...")

# Download sleep data (subject 0, night 1)
try:
    [alice_files] = sleep_physionet.age.fetch_data(
        subjects=[0], recording=[1], on_missing='warn'
    )
    raw_sleep = mne.io.read_raw_edf(alice_files[0], preload=True, verbose=False)
    annot = mne.read_annotations(alice_files[1])
    raw_sleep.set_annotations(annot)
    
    # Filter
    raw_sleep.filter(l_freq=0.5, h_freq=45, verbose=False)
    
    print(f"Loaded sleep recording: {len(raw_sleep.ch_names)} channels")
    print(f"Duration: {raw_sleep.times[-1] / 3600:.1f} hours")
    print(f"Annotations: {set(annot.description)}")
    
    SLEEP_DATA_AVAILABLE = True
except Exception as e:
    print(f"Could not load sleep data: {e}")
    print("Skipping sleep-related figures.")
    SLEEP_DATA_AVAILABLE = False

## 5. Figure 6: Sleep Stage Comparison

In [ ]:
if SLEEP_DATA_AVAILABLE:
    print("Generating Figure 6: Sleep stage comparison...")
    
    # Map sleep stages
    stage_mapping = {
        'Sleep stage W': 'Wake',
        'Sleep stage 1': 'N1',
        'Sleep stage 2': 'N2', 
        'Sleep stage 3': 'N3',
        'Sleep stage 4': 'N3',  # Combine N3/N4
        'Sleep stage R': 'REM',
        'Sleep stage ?': 'Unknown'
    }
    
    # Extract epochs for each stage
    events, event_id = mne.events_from_annotations(
        raw_sleep, event_id=stage_mapping, verbose=False
    )
    
    # Use single EEG channel
    raw_sleep.pick_channels(['EEG Fpz-Cz'])
    
    # Create epochs (30-second windows)
    epochs = mne.Epochs(raw_sleep, events, event_id, tmin=0, tmax=30,
                        baseline=None, preload=True, verbose=False)
    
    # Compute PSD for each stage
    fig, axes = plt.subplots(2, 3, figsize=(12, 7))
    
    stages_to_plot = ['Wake', 'N1', 'N2', 'N3', 'REM']
    colors = {'Wake': 'forestgreen', 'N1': 'skyblue', 'N2': 'steelblue', 
              'N3': 'navy', 'REM': 'orange'}
    
    entropies = []
    delta_powers = []
    
    for idx, stage in enumerate(stages_to_plot):
        if stage in epochs.event_id:
            stage_data = epochs[stage].get_data()
            
            if len(stage_data) > 0:
                ax = axes.flat[idx]
                
                # Compute PSD
                psds = []
                for epoch in stage_data[:min(10, len(stage_data))]:
                    f, psd = signal.welch(epoch.flatten(), 
                                          fs=raw_sleep.info['sfreq'], 
                                          nperseg=512)
                    psds.append(psd)
                
                mean_psd = np.mean(psds, axis=0)
                
                # Plot
                ax.semilogy(f, mean_psd, color=colors[stage], linewidth=2)
                ax.fill_between(f, mean_psd, alpha=0.3, color=colors[stage])
                ax.axvspan(0.5, 4, alpha=0.2, color='red')  # Delta band
                ax.axvspan(8, 13, alpha=0.2, color='green')  # Alpha band
                ax.set_xlim(0, 45)
                ax.set_xlabel('Frequency (Hz)')
                ax.set_ylabel('PSD')
                ax.set_title(f'({chr(65+idx)}) {stage}')
                
                # Compute spectral entropy (proxy for mode entropy)
                norm_psd = mean_psd / mean_psd.sum()
                norm_psd = norm_psd[norm_psd > 0]
                H = -np.sum(norm_psd * np.log(norm_psd))
                entropies.append(H)
                
                # Delta power
                delta_mask = (f >= 0.5) & (f <= 4)
                delta_power = mean_psd[delta_mask].sum() / mean_psd.sum()
                delta_powers.append(delta_power)
                
                ax.text(0.95, 0.95, f'$H = {H:.2f}$\n$\\delta = {delta_power:.2f}$',
                        transform=ax.transAxes, ha='right', va='top',
                        fontsize=9, bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Summary panel
    ax = axes[1, 2]
    x = np.arange(len(stages_to_plot))
    width = 0.35
    
    if len(entropies) == len(stages_to_plot):
        ax.bar(x - width/2, entropies, width, label='Spectral Entropy', color='steelblue')
        ax2 = ax.twinx()
        ax2.bar(x + width/2, delta_powers, width, label='Delta Power', color='coral')
        
        ax.set_ylabel('Spectral Entropy', color='steelblue')
        ax2.set_ylabel('Delta Power', color='coral')
        ax.set_xticks(x)
        ax.set_xticklabels(stages_to_plot)
        ax.set_title('(F) Summary Across Stages')
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'fig6_sleep_stages.png')
    plt.show()
    print("Saved: fig6_sleep_stages.png")
else:
    print("Skipping Figure 6 (sleep data not available)")

## 6. Figure 7: Frequency Band vs. Mode Analysis Comparison

In [ ]:
print("Generating Figure 7: Band vs. Mode analysis comparison...")

# Use sample dataset
data, times = raw[:, :int(30 * raw.info['sfreq'])]  # 30 seconds

# Traditional band analysis
bands = {
    'Delta (1-4 Hz)': (1, 4),
    'Theta (4-8 Hz)': (4, 8),
    'Alpha (8-13 Hz)': (8, 13),
    'Beta (13-30 Hz)': (13, 30),
    'Gamma (30-45 Hz)': (30, 45)
}

# Compute band powers
f, psd = signal.welch(data.mean(axis=0), fs=raw.info['sfreq'], nperseg=1024)

band_powers = {}
for band_name, (fmin, fmax) in bands.items():
    mask = (f >= fmin) & (f <= fmax)
    band_powers[band_name] = psd[mask].sum()

total_power = sum(band_powers.values())
band_powers = {k: v/total_power for k, v in band_powers.items()}

# Mode analysis
mode_amplitudes = project_to_modes(data, eigenvectors)
mode_power = compute_mode_power(mode_amplitudes)

# Create figure
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Panel A: Traditional band power
ax = axes[0, 0]
colors_bands = ['navy', 'steelblue', 'forestgreen', 'orange', 'coral']
ax.bar(range(len(band_powers)), list(band_powers.values()), 
       color=colors_bands, alpha=0.7, edgecolor='black', linewidth=0.5)
ax.set_xticks(range(len(band_powers)))
ax.set_xticklabels([k.split()[0] for k in band_powers.keys()], rotation=45, ha='right')
ax.set_ylabel('Relative Power')
ax.set_title('(A) Traditional Band Power Analysis')

# Panel B: Harmonic mode power
ax = axes[0, 1]
ax.bar(range(len(mode_power)), mode_power, color='steelblue', alpha=0.7)
ax.set_xlabel('Mode index $k$')
ax.set_ylabel('Normalized Power $p_k$')
ax.set_title('(B) Harmonic Mode Power Analysis')

# Panel C: What bands miss - spatial structure
ax = axes[1, 0]
ax.text(0.5, 0.9, 'Frequency Band Analysis:', fontsize=11, ha='center', 
        transform=ax.transAxes, fontweight='bold')
ax.text(0.5, 0.75, r'$P_{\delta} = \int_1^4 S(f) df$', fontsize=12, ha='center',
        transform=ax.transAxes)
ax.text(0.5, 0.55, 'Ignores spatial structure', fontsize=10, ha='center',
        transform=ax.transAxes, style='italic', color='darkred')
ax.text(0.5, 0.35, 'Harmonic Mode Analysis:', fontsize=11, ha='center',
        transform=ax.transAxes, fontweight='bold')
ax.text(0.5, 0.2, r'$a_k(t) = \psi_k^T X(t)$', fontsize=12, ha='center',
        transform=ax.transAxes)
ax.text(0.5, 0.0, 'Captures spatiotemporal organization', fontsize=10, ha='center',
        transform=ax.transAxes, style='italic', color='darkgreen')
ax.axis('off')
ax.set_title('(C) Conceptual Comparison')

# Panel D: Information content comparison
ax = axes[1, 1]

# Band entropy
band_probs = np.array(list(band_powers.values()))
band_entropy = -np.sum(band_probs * np.log(band_probs + 1e-10))

# Mode entropy
mode_entropy = compute_mode_entropy(mode_power)

# Mode PR
mode_pr = compute_participation_ratio(mode_power)

metrics = ['Band Entropy', 'Mode Entropy', 'Mode PR']
values = [band_entropy, mode_entropy, mode_pr / 10]  # Scale PR for visualization
colors_metrics = ['coral', 'steelblue', 'forestgreen']

bars = ax.bar(metrics, values, color=colors_metrics, alpha=0.7, edgecolor='black')
ax.set_ylabel('Value')
ax.set_title('(D) Information Metrics')

# Add actual values as text
for bar, val, true_val in zip(bars, values, [band_entropy, mode_entropy, mode_pr]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
            f'{true_val:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'fig7_band_vs_mode.png')
plt.show()
print("Saved: fig7_band_vs_mode.png")

## 7. Figure 8: Time-Varying Consciousness Functional

In [ ]:
print("Generating Figure 8: Time-varying consciousness functional...")

# Get longer data segment
data, times = raw[:, :int(60 * raw.info['sfreq'])]  # 60 seconds

# Sliding window analysis
window_size = int(2 * raw.info['sfreq'])  # 2 second windows
step_size = int(0.5 * raw.info['sfreq'])  # 0.5 second steps
n_windows = (data.shape[1] - window_size) // step_size + 1

# Initialize arrays
H_mode_series = np.zeros(n_windows)
PR_series = np.zeros(n_windows)
R_series = np.zeros(n_windows)  # Phase coherence
window_times = np.zeros(n_windows)

for i in range(n_windows):
    start = i * step_size
    end = start + window_size
    window_data = data[:, start:end]
    
    # Project to modes
    mode_amps = project_to_modes(window_data, eigenvectors)
    power = compute_mode_power(mode_amps)
    
    # Compute metrics
    H_mode_series[i] = compute_mode_entropy(power)
    PR_series[i] = compute_participation_ratio(power)
    
    # Phase coherence (using Hilbert transform)
    analytic = signal.hilbert(mode_amps[:10, :], axis=1)  # First 10 modes
    phases = np.angle(analytic)
    R = np.abs(np.mean(np.exp(1j * phases), axis=0))
    R_series[i] = np.mean(R)
    
    window_times[i] = times[start + window_size // 2]

# Normalize
H_max = np.log(len(eigenvectors))
PR_max = len(eigenvectors)

H_norm = H_mode_series / H_max
PR_norm = PR_series / PR_max

# Compute C(t) - simplified version (missing entropy production and criticality)
# Using equal weights for available components
C_t = (H_norm + PR_norm + R_series) / 3

# Create figure
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

ax = axes[0]
ax.plot(window_times, H_norm, color='steelblue', linewidth=1.5)
ax.fill_between(window_times, H_norm, alpha=0.3, color='steelblue')
ax.set_ylabel('$H_{mode}/H_{max}$')
ax.set_title('(A) Normalized Mode Entropy')
ax.set_ylim(0, 1)

ax = axes[1]
ax.plot(window_times, PR_norm, color='coral', linewidth=1.5)
ax.fill_between(window_times, PR_norm, alpha=0.3, color='coral')
ax.set_ylabel('$PR/PR_{max}$')
ax.set_title('(B) Normalized Participation Ratio')
ax.set_ylim(0, 1)

ax = axes[2]
ax.plot(window_times, R_series, color='forestgreen', linewidth=1.5)
ax.fill_between(window_times, R_series, alpha=0.3, color='forestgreen')
ax.set_ylabel('$R(t)$')
ax.set_title('(C) Phase Coherence')
ax.set_ylim(0, 1)

ax = axes[3]
ax.plot(window_times, C_t, color='purple', linewidth=2)
ax.fill_between(window_times, C_t, alpha=0.3, color='purple')
ax.set_xlabel('Time (s)')
ax.set_ylabel('$C(t)$')
ax.set_title('(D) Consciousness Functional (Simplified)')
ax.set_ylim(0, 1)

# Add mean lines
for ax, series in zip(axes, [H_norm, PR_norm, R_series, C_t]):
    ax.axhline(y=np.mean(series), color='black', linestyle='--', alpha=0.5)
    ax.text(0.02, 0.95, f'Mean = {np.mean(series):.3f}',
            transform=ax.transAxes, va='top', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'fig8_time_varying_C.png')
plt.show()
print("Saved: fig8_time_varying_C.png")

## Summary

This notebook generated the following real-data figures:

1. **fig5_real_eeg_harmonics.png** - EEG sensor layout, Laplacian eigenspectrum, eigenmodes, mode power distribution
2. **fig6_sleep_stages.png** - Sleep stage comparison (Wake, N1, N2, N3, REM) with spectral entropy and delta power
3. **fig7_band_vs_mode.png** - Comparison of traditional frequency band analysis vs. harmonic mode analysis
4. **fig8_time_varying_C.png** - Time-varying components of the consciousness functional

These figures complement the synthetic figures from `generate_figures.py` by demonstrating the harmonic field model applied to real neurophysiological data.